In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from scipy.optimize import curve_fit

import sys
sys.path.append("../src")

from data_prep import *
from make_cuts import *
from fitting_functions import *
from pulse_profile_generator import *
from plot_maker import *

In [36]:
pd.set_option('display.max_columns', None)

In [37]:
c_folder="../../data/calibration/"
d_folder="../../data/runData/"
r_n=287
pulsed=True

Path("../../plots").mkdir(parents=True, exist_ok=True)
Path("../../plots/run_"+str(r_n)).mkdir(parents=True, exist_ok=True)
p_folder="../../plots/run_"+str(r_n)+"/"

In [38]:
%%time
p,m,r=import_data(c_folder,d_folder, r_n, is_it_pulsed=pulsed)

CPU times: user 4.21 s, sys: 1.19 s, total: 5.4 s
Wall time: 8.7 s


In [39]:
%%time
r=select_bars_with_data(r,p, subtract_pedestal=True, is_it_pulsed=pulsed) #this needs to be run as it's own cell

Number of events: 7800


Initial pedestal-based selection performed.
Events left: 7764


CPU times: user 345 ms, sys: 314 ms, total: 658 ms
Wall time: 1.86 s


In [40]:
# %%time 
# make cuts
# r=first_layer_cut(r)
# r=back_layer_cut(r,5)

In [41]:
# %%time
# fit gaussian to distribution of reconstructed energy
# p0,p1=make_gaussian_fit_plot(r,m,r_n,p_folder, save_plot=False)

In [42]:
# %%time # make shower profiles
# make_shower_profiles(r, m, r_n, p_folder)

In [43]:
%%time 
# make the pulse profile dataframe
pulses=make_pulse_profiles(r)
pulses.to_csv(d_folder+"/run_"+str(r_n)+"_pulse_problems.csv")

CPU times: user 21.9 s, sys: 95.3 ms, total: 22 s
Wall time: 22.1 s
